In [1]:
import numpy as np
import heapq
import pandas as pd
import matplotlib.pyplot as plt
import random
import seaborn as sns

In [2]:
#normalize
df = pd.read_excel("Database.xlsx")
index = ["Treatment name","Treatment type1","n randomized in this arm","Total sample size"]
data = pd.DataFrame(df,columns = index)[(df["Treatment type1"] !='*') & (df["n randomized in this arm"] != "*")]
data.index = range(len(data))
values = [[],[]]
for i in [0,1] : 
    data[index[i]] = data[index[i]].str.lower()
    data[index[i]] = data[index[i]].str.replace(', ','+')
    values[i] = data[index[i]].unique()
#print(data.shape,len(data[index[i]]))

In [3]:
#list of all kinds of treatments
#print(len(values[0]),len(values[1]))
all_values = values[0]
for temp in values[1] : 
    if not temp in all_values :
        all_values = np.append(all_values, [temp], axis = 0)
l = len(all_values)

In [4]:
#calculer le nombre d'expériences faites pour comparer deux traitements différents
patients_nb = np.eye(l)
for i in range(0,462) :
    treatmt1 = data.loc[i,index[0]]
    po1 = np.where(all_values == treatmt1)
    treatmt2 = data.loc[i,index[1]]
    po2 = np.where(all_values == treatmt2)
    #nb = data.loc[i,index[2]]
    patients_nb[po1,po2] = patients_nb[po1,po2] + 1
#print(patients_nb.max())
#print(patients_nb[6,194],all_values[6],all_values[194])# les traitements les plus comparés
patients_nb = patients_nb + np.transpose(patients_nb) - 2*np.eye(239)

m = []#la liste des 10 traitements les plus comparés
temp = np.array(patients_nb, copy = True)
while (len(m)<10):
    maxi = temp.max()
    po = np.where(temp == maxi)#po est un array de deux positions car la matrice est symetrique, on prend juste le premier
    print(po)
    temp[po[0]] = 0
    print(po[0])
    m.append(po[0])
print(m)

# on cree une liste avec les noms de traitements les plus comparés


(array([  6, 194]), array([194,   6]))
[  6 194]
(array([  3, 193]), array([193,   3]))
[  3 193]
(array([32]), array([194]))
[32]
(array([119, 203]), array([203, 119]))
[119 203]
(array([17]), array([203]))
[17]
(array([ 27, 190]), array([190,  27]))
[ 27 190]
(array([ 28, 196]), array([196,  28]))
[ 28 196]
(array([ 0, 97]), array([190, 196]))
[ 0 97]
(array([ 13, 199]), array([199,  13]))
[ 13 199]
(array([ 12,  18, 205]), array([193, 205,  18]))
[ 12  18 205]
[array([  6, 194]), array([  3, 193]), array([32]), array([119, 203]), array([17]), array([ 27, 190]), array([ 28, 196]), array([ 0, 97]), array([ 13, 199]), array([ 12,  18, 205])]


In [11]:
m = []#la liste des 10 traitements les plus comparés
temp = np.array(patients_nb, copy = True)
while (len(m)<10):
    maxi = temp.max()
    po1 = np.where(temp == maxi)[0]
    po2 = np.where(temp == maxi)[1]#po est un array de deux positions car la matrice est symetrique, on prend juste le premier
    temp[po1,po2] = 0
    m.append(po1)
print(m)

[array([  6, 194]), array([  3, 193]), array([ 32, 194]), array([119, 203]), array([ 17, 203]), array([ 27, 190]), array([ 28, 196]), array([  0,  97, 190, 196]), array([ 13, 199]), array([ 12,  18, 193, 205])]


In [7]:
m = []#la liste des 10 traitements les plus comparés
temp = np.array(patients_nb, copy = True)
while (len(m)<10):
    po = np.argwhere(temp == temp.max())
    temp[po] = 0
    print(po)
    m.append(po[0])
print(m)

[[  6 194]
 [194   6]]
[[  3 193]
 [193   3]]
[[ 32 194]]
[[119 203]
 [203 119]]
[[ 17 203]]
[[ 27 190]
 [190  27]]
[[ 28 196]
 [196  28]]
[[  0 190]
 [ 97 196]]
[[ 13 199]
 [199  13]]
[[ 12 193]
 [ 18 205]
 [205  18]]
[array([  6, 194]), array([  3, 193]), array([ 32, 194]), array([119, 203]), array([ 17, 203]), array([ 27, 190]), array([ 28, 196]), array([  0, 190]), array([ 13, 199]), array([ 12, 193])]


In [ ]:
#on ajoute une ligne des indices correspondants avant de supprimer tous les 0 pour que les indices ne se confondent pas 
temp1 = np.arange(239)
patients_nb_ind = np.vstack((patients_nb,temp1))
temp2 = np.transpose(np.arange(239))
temp2 = np.hstack((temp2,0))
patients_nb_ind = np.column_stack((patients_nb_ind,temp2))
patients_nb_ind = patients_nb_ind - 2*np.eye(240)# on enlève les valeurs dans le diagonale
#print(patients_nb_ind)
#on enlève tous les 0,1,2
ind = []# les caisses qu'on doit enlever
for i in range(0,239) : 
    if  patients_nb[i].max() <= 2 :
        temp = np.where(patients_nb[i] == patients_nb[i].max())
    print(temp)
    if np.count_nonzero(patients_nb_ind[i]) <= 2 :
        ind.append(i)
patients_np_ind = np.delete(patients_nb_ind,ind,axis=0)

a = np.transpose(patients_np_ind)
patients_np_ind = np.delete(a,ind,axis=0)
patients_nb_ind = patients_np_ind

#on enleve les indices
print(patients_nb_ind[0:35,0:35])

sns.set()
np.random.seed(0)
ax = sns.heatmap(patients_nb_ind[0:35,0:35])
plt.savefig('cluster.tif',dpi = 300)
plt.show()